In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from mp_api.client import MPRester
from pymatgen.core import Composition, Element
from pymatgen.analysis.phase_diagram import GrandPotentialPhaseDiagram, PhaseDiagram
from pymatgen.analysis.interface_reactions import InterfacialReactivity, GrandPotentialInterfacialReactivity
from pymatgen.io.vasp import Vasprun
from pymatgen.entries.compatibility import MaterialsProject2020Compatibility
from pymatgen.entries.computed_entries import ComputedEntry

    

In [5]:
api_key = 'HbSHn48X1CSvBzRa4DOX5E9rhb5Tcc03'

In [39]:
def create_dataframe_from_calc(vasprun_path: str, calc_name: str, if_save: bool) -> None:
    """
        Inputs:
            vasprun_path - path to vasprun.xml
            calc_name - name to save a file
            if_save - save pandas dataframe as csv
        Returns:
            pandasDataframe - dataframe with the results
    """

    # Extraction of the calculation
    vasprun = Vasprun(vasprun_path)    
    # include structure so proper correction can be applied for oxides and sulfides
    reactant2 = vasprun.get_computed_entry(inc_structure=True)
    reactant2.parameters['run_type'] = 'GGA+U'
    compatibility = MaterialsProject2020Compatibility()
    compatibility.process_entry(reactant2)

    # Find coatings 
    comps = []
    emins = []
    reactions = []
    
    with MPRester(api_key=api_key) as mpr:
        for comp in tqdm(cathode_coatings.formula_pretty.unique()):
            try:
                # Is the system open to an elemental reservoir?
                grand = True
    
                if grand:
                    # Element in the elemental reservoir.
                    open_el = "Li" 
                    # Relative chemical potential vs. pure substance. Must be non-positive.
                    relative_mu = -3.8 # average intercalation potential
    
    
                # Get the compositions of the reactants
                reactant1 = comp
    
                comp1 = Composition(reactant1)
                comp2 = reactant2.composition # as it is manually calculated phase
    
                # Gather all elements involved in the chemical system.
                # elements = [e.symbol for e in comp1.elements + comp2.elements]
                elements = [e.symbol for e in comp1.elements + comp2.elements]
                
                if grand:
                    elements.append(open_el)
                elements = list(set(elements))  # Remove duplicates
    
                entries = mpr.get_entries_in_chemsys(elements, additional_criteria={'thermo_types': ['GGA_GGA+U']})
    
                # Build a phase diagram using these entries.
                phase_diagram = PhaseDiagram(entries)
    
                # For an open system, include the grand potential phase diagram.
                if grand:
                    # Get the chemical potential of the pure subtance.
                    mu = phase_diagram.get_transition_chempots(Element(open_el))[0]
                    # Set the chemical potential in the elemental reservoir.
                    chempots = {open_el: relative_mu + mu}
                    # Build the grand potential phase diagram
                    gpd = GrandPotentialPhaseDiagram(entries, chempots)
                    # Create InterfacialReactivity object.
                    interface = GrandPotentialInterfacialReactivity(
                        comp1,
                        comp2,
                        gpd,
                        norm=True,
                        include_no_mixing_energy=True,
                        pd_non_grand=phase_diagram,
                        use_hull_energy=True,
                    )
                    
                else:
                    interface = InterfacialReactivity(
                        comp1,
                        comp2,
                        phase_diagram,
                        norm=True,
                        include_no_mixing_energy=False,
                        pd_non_grand=None,
                        use_hull_energy=True,
                    )
                    
                interface_table = interface.get_dataframe()
                emins.append(interface.get_dataframe()['E$_{\textrm{rxn}}$ (eV/atom)'].min())
                comps.append(comp)
                index = np.argmin(interface_table['E$_{\textrm{rxn}}$ (eV/atom)'].values)
                reactions.append(interface_table['Reaction'].iloc[index])
            except:
                continue

    res = pd.DataFrame()
    res['formula_pretty'] = comps
    res[f'e_rxn_{calc_name}_CC'] = emins
    res[f'interface_reaction_{calc_name}'] = reactions

    if if_save:
        res.merge(cathode_coatings).round(3).to_csv(f'data_nmc/{calc_name}_CC_calculation_results.csv', index = False)
        
    return res
    
    
    


In [40]:
# Path to vasprun.xml file
vasprun_path = "/home/arseniy/Desktop/work/nvpf_al_doped/data/stability/vasprun_inter.xml"



In [41]:
# Parse vasprun.xml
vasprun = Vasprun(vasprun_path)

# Get final energy
final_energy = vasprun.final_energy
print(f"Final energy: {final_energy} eV")


Final energy: -905.02190536 eV eV


## DOS

In [9]:
# Access the structure
structure = vasprun.final_structure
print(structure)

# Access density of states (DOS) if included
dos = vasprun.complete_dos
print(dos)

Full Formula (Na16 V16 P16 O64 F16)
Reduced Formula: NaVPO4F
abc   :  12.845161  12.688343  10.761530
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (128)
  #  SP           a          b         c
---  ----  --------  ---------  --------
  0  Na    0.849576   0.392818  0.274601
  1  Na    0.150421   0.107181  0.774613
  2  Na    0.349581   0.357188  0.274608
  3  Na    0.650414   0.142814  0.774613
  4  Na    0.601043   0.387742  0.066647
  5  Na    0.398963   0.112265  0.566655
  6  Na    0.101031   0.362264  0.066653
  7  Na    0.898959   0.137737  0.566653
  8  Na    0.849571   0.892813  0.274609
  9  Na    0.150424   0.607188  0.774614
 10  Na    0.349582   0.857183  0.274611
 11  Na    0.650427   0.642818  0.774604
 12  Na    0.60104    0.88774   0.066645
 13  Na    0.398965   0.612266  0.566654
 14  Na    0.101037   0.862264  0.066652
 15  Na    0.898959   0.637735  0.566643
 16  V     0.883662   0.256347  0.007477
 17  V     0.116332   0.2

## Thermodynamic stability

In [42]:
from pymatgen.entries.compatibility import MaterialsProjectCompatibility


In [61]:
# Path to vasprun.xml file
vasprun_path = "/home/arseniy/Desktop/work/nvpf_al_doped/data/stability/vasprun_al.xml"



In [62]:
# Parse vasprun.xml
vasprun = Vasprun(vasprun_path)

# Get final energy
final_energy = vasprun.final_energy
print(f"Final energy: {final_energy} eV")


Final energy: -899.07621043 eV eV


/home/arseniy/micromamba/envs/default/lib/python3.12/site-packages/pymatgen/io/vasp/outputs.py:1256: UserWarning: No POTCAR file with matching TITEL fields was found in

  if potcar := self.get_potcars(path):
/home/arseniy/micromamba/envs/default/lib/python3.12/site-packages/pymatgen/io/vasp/outputs.py:1274: UserWarning: No POTCAR file with matching TITEL fields was found in

  potcar = self.get_potcars(path)


In [63]:
comp_entry = ComputedEntry(vasprun.final_structure.composition, vasprun.final_energy)


In [64]:
elements = [el.symbol for el in comp_entry.elements]  # elements in your compound

with MPRester(api_key) as mpr:
    entries = mpr.get_entries_in_chemsys(elements)  # all known phases in this system

    

Retrieving ThermoDoc documents:   0%|          | 0/1365 [00:00<?, ?it/s]

In [65]:
compat = MaterialsProjectCompatibility()
# MaterialsProject2020Compatibility
entries = compat.process_entries(entries)

# Also process your entry with the same corrections
# entry = compat.process_entry(comp_entry)


/tmp/ipykernel_19699/992659018.py:1: DeprecationWarning: MaterialsProjectCompatibility is deprecated; use MaterialsProject2020Compatibility instead.
Materials Project formation energies use the newer MaterialsProject2020Compatibility scheme.
  compat = MaterialsProjectCompatibility()


In [66]:
all_entries = entries + [comp_entry]
pd = PhaseDiagram(all_entries)

decomp, e_above_hull = pd.get_decomp_and_e_above_hull(comp_entry)

print(f"Energy above hull: {e_above_hull:.4f} eV/atom")

if e_above_hull == 0:
    print("Phase is thermodynamically stable")
elif e_above_hull < 0.05:
    print("Phase is metastable")
else:
    print("Phase is decomposes:")
    for k, v in decomp.items():
        print(f"  {k.composition.reduced_formula} : {v:.3f}")




Energy above hull: 0.3655 eV/atom
Phase is decomposes:
  Na3V2P2O8F3 : 0.750
  AlPO4 : 0.094
  VPO4 : 0.156
